# Example Predictor: Linear Rollout Predictor

This example contains basic functionality for training and evaluating a linear predictor that rolls out predictions day-by-day.

First, a training data set is created from historical case and npi data.

Second, a linear model is trained to predict future cases from prior case data along with prior and future npi data.
The model is an off-the-shelf sklearn Lasso model, that uses a positive weight constraint to enforce the assumption that increased npis has a negative correlation with future cases.

Third, a sample evaluation set is created, and the predictor is applied to this evaluation set to produce prediction results in the correct format.

## Training

In [1]:
import pickle
import numpy as np
import pandas as pd
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split

### Copy the data locally

In [2]:
# Main source for the training data
DATA_URL = 'https://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/master/data/OxCGRT_latest.csv'
# Local file
DATA_FILE = 'data/OxCGRT_latest.csv'

In [3]:
import os
import urllib.request
if not os.path.exists('data'):
    os.mkdir('data')
urllib.request.urlretrieve(DATA_URL, DATA_FILE)

('data/OxCGRT_latest.csv', <http.client.HTTPMessage at 0x17e410cf888>)

In [4]:
# Load historical data from local file
df = pd.read_csv(DATA_FILE, 
                 parse_dates=['Date'],
                 encoding="ISO-8859-1",
                 dtype={"RegionName": str,
                        "RegionCode": str},
                 error_bad_lines=False)

In [5]:
df.columns

Index(['CountryName', 'CountryCode', 'RegionName', 'RegionCode',
       'Jurisdiction', 'Date', 'C1_School closing', 'C1_Flag',
       'C2_Workplace closing', 'C2_Flag', 'C3_Cancel public events', 'C3_Flag',
       'C4_Restrictions on gatherings', 'C4_Flag', 'C5_Close public transport',
       'C5_Flag', 'C6_Stay at home requirements', 'C6_Flag',
       'C7_Restrictions on internal movement', 'C7_Flag',
       'C8_International travel controls', 'E1_Income support', 'E1_Flag',
       'E2_Debt/contract relief', 'E3_Fiscal measures',
       'E4_International support', 'H1_Public information campaigns',
       'H1_Flag', 'H2_Testing policy', 'H3_Contact tracing',
       'H4_Emergency investment in healthcare', 'H5_Investment in vaccines',
       'H6_Facial Coverings', 'H6_Flag', 'M1_Wildcard', 'ConfirmedCases',
       'ConfirmedDeaths', 'StringencyIndex', 'StringencyIndexForDisplay',
       'StringencyLegacyIndex', 'StringencyLegacyIndexForDisplay',
       'GovernmentResponseIndex', 'Gove

In [6]:
# For testing, restrict training data to that before a hypothetical predictor submission date
HYPOTHETICAL_SUBMISSION_DATE = np.datetime64("2020-07-31")
df = df[df.Date <= HYPOTHETICAL_SUBMISSION_DATE]

In [7]:
# Add RegionID column that combines CountryName and RegionName for easier manipulation of data
df['GeoID'] = df['CountryName'] + '__' + df['RegionName'].astype(str)

In [8]:
# Add new cases column
df['NewCases'] = df.groupby('GeoID').ConfirmedCases.diff().fillna(0)

In [9]:
# Keep only columns of interest
id_cols = ['CountryName',
           'RegionName',
           'GeoID',
           'Date']
cases_col = ['NewCases']
npi_cols = ['C1_School closing',
            'C2_Workplace closing',
            'C3_Cancel public events',
            'C4_Restrictions on gatherings',
            'C5_Close public transport',
            'C6_Stay at home requirements',
            'C7_Restrictions on internal movement',
            'C8_International travel controls',
            'H1_Public information campaigns',
            'H2_Testing policy',
            'H3_Contact tracing',
            'H6_Facial Coverings']
df = df[id_cols + cases_col + npi_cols]

In [10]:
# Fill any missing case values by interpolation and setting NaNs to 0
df.update(df.groupby('GeoID').NewCases.apply(
    lambda group: group.interpolate()).fillna(0))

In [11]:
# Fill any missing NPIs by assuming they are the same as previous day
for npi_col in npi_cols:
    df.update(df.groupby('GeoID')[npi_col].ffill().fillna(0))

In [12]:
df

,CountryName,RegionName,GeoID,Date,NewCases,C1_School closing,C2_Workplace closing,C3_Cancel public events,C4_Restrictions on gatherings,C5_Close public transport,C6_Stay at home requirements,C7_Restrictions on internal movement,C8_International travel controls,H1_Public information campaigns,H2_Testing policy,H3_Contact tracing,H6_Facial Coverings
0,Aruba,NaN,Aruba__nan,2020-01-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Aruba,NaN,Aruba__nan,2020-01-02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Aruba,NaN,Aruba__nan,2020-01-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Aruba,NaN,Aruba__nan,2020-01-04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Aruba,NaN,Aruba__nan,2020-01-05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87064,Zimbabwe,NaN,Zimbabwe__nan,2020-07-27,78.0,3.0,1.0,2.0,3.0,1.0,2.0,2.0,4.0,2.0,1.0,1.0,4.0
87065,Zimbabwe,NaN,Zimbabwe__nan,2020-07-28,192.0,3.0,1.0,2.0,3.0,1.0,2.0,2.0,4.0,2.0,1.0,1.0,4.0
87066,Zimbabwe,NaN,Zimbabwe__nan,2020-07-29,113.0,3.0,1.0,2.0,3.0,1.0,2.0,2.0,4.0,2.0,1.0,1.0,4.0
87067,Zimbabwe,NaN,Zimbabwe__nan,2020-07-30,62.0,3.0,1.0,2.0,3.0,1.0,2.0,2.0,4.0,2.0,1.0,1.0,4.0


In [13]:
# Set number of past days to use to make predictions
nb_lookback_days = 30

# Create training data across all countries for predicting one day ahead
X_cols = cases_col + npi_cols
y_col = cases_col
X_samples = []
y_samples = []
geo_ids = df.GeoID.unique()
for g in geo_ids:
    gdf = df[df.GeoID == g]
    all_case_data = np.array(gdf[cases_col])
    all_npi_data = np.array(gdf[npi_cols])

    # Create one sample for each day where we have enough data
    # Each sample consists of cases and npis for previous nb_lookback_days
    nb_total_days = len(gdf)
    for d in range(nb_lookback_days, nb_total_days - 1):
        X_cases = all_case_data[d-nb_lookback_days:d]

        # Take negative of npis to support positive
        # weight constraint in Lasso.
        X_npis = -all_npi_data[d - nb_lookback_days:d]

        # Flatten all input data so it fits Lasso input format.
        X_sample = np.concatenate([X_cases.flatten(),
                                   X_npis.flatten()])
        y_sample = all_case_data[d + 1]
        X_samples.append(X_sample)
        y_samples.append(y_sample)

X_samples = np.array(X_samples)
y_samples = np.array(y_samples).flatten()

In [14]:
# Helpful function to compute mae
def mae(pred, true):
    return np.mean(np.abs(pred - true))

In [15]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_samples,
                                                    y_samples,
                                                    test_size=0.2,
                                                    random_state=301)

In [16]:
# Create and train Lasso model.
# Set positive=True to enforce assumption that cases are positively correlated
# with future cases and npis are negatively correlated.
model = Lasso(alpha=0.1,
              precompute=True,
              max_iter=10000,
              positive=True,
              selection='random')
# Fit model
model.fit(X_train, y_train)

Lasso(alpha=0.1, copy_X=True, fit_intercept=True, max_iter=10000,
      normalize=False, positive=True, precompute=True, random_state=None,
      selection='random', tol=0.0001, warm_start=False)

In [17]:
# Evaluate model
train_preds = model.predict(X_train)
train_preds = np.maximum(train_preds, 0) # Don't predict negative cases
print('Train MAE:', mae(train_preds, y_train))

test_preds = model.predict(X_test)
test_preds = np.maximum(test_preds, 0) # Don't predict negative cases
print('Test MAE:', mae(test_preds, y_test))

Train MAE: 140.71142554508796
Test MAE: 152.4961604640549


In [18]:
# Inspect the learned feature coefficients for the model
# to see what features it's paying attention to.

# Give names to the features
x_col_names = []
for d in range(-nb_lookback_days, 0):
    x_col_names.append('Day ' + str(d) + ' ' + cases_col[0])
for d in range(-nb_lookback_days, 1):
    for col_name in npi_cols:
        x_col_names.append('Day ' + str(d) + ' ' + col_name)

# View non-zero coefficients
for (col, coeff) in zip(x_col_names, list(model.coef_)):
    if coeff != 0.:
        print(col, coeff)
print('Intercept', model.intercept_)

Day -7 NewCases 0.0011608475677650214
Day -6 NewCases 0.43952450027599643
Day -5 NewCases 0.21711030732343317
Day -4 NewCases 0.059028719919464595
Day -3 NewCases 0.06940508018514298
Day -2 NewCases 0.05198015541172006
Day -1 NewCases 0.23835895029729234
Day -26 C6_Stay at home requirements 4.315284773631653
Day -22 C2_Workplace closing 9.716177378999028
Day -17 C2_Workplace closing 5.765630705654963
Intercept 26.556349932951832


In [19]:
# Save model to file
if not os.path.exists('models'):
    os.mkdir('models')
with open('models/model.pkl', 'wb') as model_file:
    pickle.dump(model, model_file)

## Evaluation

Now that the predictor has been trained and saved, this section contains the functionality for evaluating it on sample evaluation data.

In [20]:
# Reload the module to get the latest changes
import predict
from importlib import reload
reload(predict)
from predict import predict_df

In [21]:
list_countries = sorted(list(set(df.CountryName)))
hist_ips_df = pd.read_csv("data/2020-09-30_historical_ip.csv",
                              parse_dates=['Date'],
                              encoding="ISO-8859-1",
                              dtype={"RegionName": str},
                              error_bad_lines=True)
hist_ips_df = hist_ips_df[hist_ips_df.CountryName.isin(list_countries)]
hist_ips_df.to_csv("data/2020-09-30_historical_ip_new.csv" , index = False)

In [22]:
%%time
preds_df = predict_df("2020-08-01", "2020-08-31", path_to_ips_file="data/2020-09-30_historical_ip_new.csv", verbose=True)


Predicting for Aruba__nan
2020-08-01: 58.83415407877576
2020-08-02: 71.32515221885238
2020-08-03: 78.99290276184884
2020-08-04: 90.24148802296129
2020-08-05: 87.81474353574718
2020-08-06: 98.76529261660889
2020-08-07: 131.04548858897294
2020-08-08: 147.02767952954667
2020-08-09: 158.958731678789
2020-08-10: 169.946226662304
2020-08-11: 177.52399757978435
2020-08-12: 193.49142350203726
2020-08-13: 216.82870610844768
2020-08-14: 234.0482504086842
2020-08-15: 248.56932611676996
2020-08-16: 261.97622729928634
2020-08-17: 275.3094472671903
2020-08-18: 293.3023526062957
2020-08-19: 314.08623682222384
2020-08-20: 332.44049891970025
2020-08-21: 349.24472728944994
2020-08-22: 365.5131260236332
2020-08-23: 382.54734140390616
2020-08-24: 407.9047871685246
2020-08-25: 430.0962569233913
2020-08-26: 450.5860861516399
2020-08-27: 466.0129087227012
2020-08-28: 484.66028347426965
2020-08-29: 515.3663534295372
2020-08-30: 541.9178913415958
2020-08-31: 566.2793263685082

Predicting for Afghanistan__nan


2020-08-21: 7910.986024204321
2020-08-22: 7860.781264957484
2020-08-23: 7808.159044090948
2020-08-24: 7994.6898787791715
2020-08-25: 8369.320083567198
2020-08-26: 8630.511544506488
2020-08-27: 8742.821056496457
2020-08-28: 8786.793996076804
2020-08-29: 8860.665892848101
2020-08-30: 9067.032153550104
2020-08-31: 9351.325749395939

Predicting for Azerbaijan__nan
2020-08-01: 2144.0217949319003
2020-08-02: 2777.044526099755
2020-08-03: 3002.3690737418406
2020-08-04: 2959.1302554376925
2020-08-05: 2639.26260112625
2020-08-06: 1675.0809907652488
2020-08-07: 2515.0083286818744
2020-08-08: 2969.979608882416
2020-08-09: 3126.668068017902
2020-08-10: 3100.8569432371796
2020-08-11: 2834.032844518544
2020-08-12: 2565.026737903932
2020-08-13: 2961.3231926289277
2020-08-14: 3256.7232064271375
2020-08-15: 3377.1060273197554
2020-08-16: 3363.6881974824732
2020-08-17: 3234.9328995501833
2020-08-18: 3208.3645148261076
2020-08-19: 3439.5178743160486
2020-08-20: 3639.9379430403437
2020-08-21: 3740.6219709

2020-08-01: 1226.6276183159891
2020-08-02: 1258.2139362985847
2020-08-03: 1286.0992825082685
2020-08-04: 1238.7120274941826
2020-08-05: 880.8257797488732
2020-08-06: 755.6662321794918
2020-08-07: 1250.7678056481016
2020-08-08: 1355.9986359303377
2020-08-09: 1379.0092256515513
2020-08-10: 1318.442194925048
2020-08-11: 1157.2021500010464
2020-08-12: 1175.4950673010653
2020-08-13: 1408.9393697152652
2020-08-14: 1502.5898064925234
2020-08-15: 1525.8846736044718
2020-08-16: 1491.9863481218326
2020-08-17: 1438.4293057104735
2020-08-18: 1489.582487057113
2020-08-19: 1620.9719532264976
2020-08-20: 1695.7208636288156
2020-08-21: 1723.744170160842
2020-08-22: 1719.9479506035282
2020-08-23: 1720.9703494744786
2020-08-24: 1778.3209239324951
2020-08-25: 1867.4718235164974
2020-08-26: 1930.6403531536703
2020-08-27: 1957.320921249188
2020-08-28: 1975.1228123889903
2020-08-29: 2003.295956263569
2020-08-30: 2061.080895379176
2020-08-31: 2132.09447741659

Predicting for Belarus__nan
2020-08-01: 1205.022

2020-08-23: 390.72664663245297
2020-08-24: 415.87203695531144
2020-08-25: 438.89261167106304
2020-08-26: 460.3701416023775
2020-08-27: 476.8973117653649
2020-08-28: 498.74554704719833
2020-08-29: 525.8083651202487
2020-08-30: 551.8930413018284
2020-08-31: 576.8195327320223

Predicting for Botswana__nan
2020-08-01: 157.26988846067482
2020-08-02: 303.96194272724205
2020-08-03: 131.8656665632026
2020-08-04: 108.81540766865888
2020-08-05: 113.84019496029461
2020-08-06: 144.6993441003789
2020-08-07: 241.529985886933
2020-08-08: 292.4960606827778
2020-08-09: 231.6412496043434
2020-08-10: 219.0873017805376
2020-08-11: 231.06640539924712
2020-08-12: 266.6461204298091
2020-08-13: 324.9465726427689
2020-08-14: 350.0838509634558
2020-08-15: 332.868928876142
2020-08-16: 333.2311864106621
2020-08-17: 350.58401446990484
2020-08-18: 383.337756346538
2020-08-19: 422.1792162487164
2020-08-20: 441.7443190789353
2020-08-21: 444.2658632481009
2020-08-22: 454.41981404399115
2020-08-23: 475.36043711517146
2

2020-08-04: 139.70429446310726
2020-08-05: 134.9006722097803
2020-08-06: 133.16292576592227
2020-08-07: 187.72001449684083
2020-08-08: 220.2956431519963
2020-08-09: 230.8822229129866
2020-08-10: 228.91902551516083
2020-08-11: 231.97079862243658
2020-08-12: 246.3293767187623
2020-08-13: 281.4488249915752
2020-08-14: 307.3417194207196
2020-08-15: 320.7803835883265
2020-08-16: 328.4265502048696
2020-08-17: 339.27422617547495
2020-08-18: 358.65773545271185
2020-08-19: 386.23992254983267
2020-08-20: 415.13080180996707
2020-08-21: 433.03330771791275
2020-08-22: 447.5922303405837
2020-08-23: 464.61142493257177
2020-08-24: 486.8932906733539
2020-08-25: 523.2904917349895
2020-08-26: 551.7820048927194
2020-08-27: 569.7639754394238
2020-08-28: 589.4873808543507
2020-08-29: 611.5841490383752
2020-08-30: 638.5215849481347
2020-08-31: 670.7304432124315

Predicting for Congo__nan
2020-08-01: 57.575087795446485
2020-08-02: 89.79417101190788
2020-08-03: 126.48910045382537
2020-08-04: 89.61787448789536


...

2020-08-31: 6523.546807811873

Predicting for United States__Mississippi
2020-08-01: 1228.0866407993203
2020-08-02: 1587.5937338575254
2020-08-03: 1571.955825402438
2020-08-04: 1089.5561707547886
2020-08-05: 882.8960795252972
2020-08-06: 788.0072558126888
2020-08-07: 1337.2587894095723
2020-08-08: 1576.469423998673
2020-08-09: 1532.063104535825
2020-08-10: 1309.5196914482176
2020-08-11: 1191.1967704707338
2020-08-12: 1239.7661809889214
2020-08-13: 1520.3602480116222
2020-08-14: 1664.5537984442037
2020-08-15: 1642.3390111496126
2020-08-16: 1543.3268076493523
2020-08-17: 1503.4232661341894
2020-08-18: 1583.6305575908557
2020-08-19: 1747.1814855377977
2020-08-20: 1840.599694231374
2020-08-21: 1843.4863750829336
2020-08-22: 1812.908585910011
2020-08-23: 1831.384531340537
2020-08-24: 1910.6290476545373
2020-08-25: 2020.7860173746953
2020-08-26: 2092.5155933963706
2020-08-27: 2112.3527129458257
2020-08-28: 2123.707719803996
2020-08-29: 2164.2161815855675
2020-08-30: 2238.8404406996683
2020-0

2020-08-11: 2173.0072761777915
2020-08-12: 2011.4004161086705
2020-08-13: 2325.430264271118
2020-08-14: 2491.3286328007985
2020-08-15: 2572.505814448265
2020-08-16: 2587.5513074846535
2020-08-17: 2505.863439172439
2020-08-18: 2505.2927410318466
2020-08-19: 2680.6010376327695
2020-08-20: 2808.4815785202504
2020-08-21: 2882.352537394275
2020-08-22: 2907.713009748653
2020-08-23: 2910.5274317305657
2020-08-24: 2962.9077353224407
2020-08-25: 3086.4758098506895
2020-08-26: 3192.7927177314095
2020-08-27: 3262.1662641804014
2020-08-28: 3307.739975558585
2020-08-29: 3349.5206749368767
2020-08-30: 3422.792631227144
2020-08-31: 3527.1420020960895

Predicting for United States__New York
2020-08-01: 4611.584640717378
2020-08-02: 5584.295368129514
2020-08-03: 5772.139037255095
2020-08-04: 5695.513470887256
2020-08-05: 4970.166257916438
2020-08-06: 3269.734119253876
2020-08-07: 5063.721200450069
2020-08-08: 5821.7458004872815
2020-08-09: 6001.899392748317
2020-08-10: 5917.4392229500945
2020-08-11: 53

Predicting for United States__Tennessee
2020-08-01: 2859.832309972857
2020-08-02: 3748.5635816531967
2020-08-03: 4247.756298918712
2020-08-04: 4363.038083388452
2020-08-05: 3536.3684778160114
2020-08-06: 2262.1023444411885
2020-08-07: 3398.0568339524443
2020-08-08: 4054.33394962037
2020-08-09: 4378.040747930873
2020-08-10: 4364.705424028095
2020-08-11: 3851.0920326116657
2020-08-12: 3474.7772113416668
2020-08-13: 4016.849087232896
2020-08-14: 4450.109945724189
2020-08-15: 4665.266098324398
2020-08-16: 4637.4844310116805
2020-08-17: 4396.651609794357
2020-08-18: 4335.769831489287
2020-08-19: 4651.3942150264475
2020-08-20: 4942.877173499831
2020-08-21: 5099.357056101436
2020-08-22: 5105.870680576243
2020-08-23: 5045.032351430403
2020-08-24: 5100.422638614525
2020-08-25: 5322.089838991022
2020-08-26: 5536.4210865878285
2020-08-27: 5665.498311663127
2020-08-28: 5715.896361159529
2020-08-29: 5747.872477448999
2020-08-30: 5852.125282014015
2020-08-31: 6033.780126791267

Predicting for United

2020-08-16: 1246.6779601788999
2020-08-17: 1213.5885489392135
2020-08-18: 1256.933130827866
2020-08-19: 1369.395911939398
2020-08-20: 1437.4313787514225
2020-08-21: 1458.1105534449466
2020-08-22: 1455.092576482973
2020-08-23: 1461.636849613967
2020-08-24: 1511.920569970519
2020-08-25: 1589.5091846128698
2020-08-26: 1645.4164081078761
2020-08-27: 1670.849173220672
2020-08-28: 1688.288953952435
2020-08-29: 1721.7836924484334
2020-08-30: 1774.6930925438287
2020-08-31: 1838.0557595595046

Predicting for United States__Wyoming
2020-08-01: 787.8482248302926
2020-08-02: 854.8407273899502
2020-08-03: 659.2819052238974
2020-08-04: 914.8483952422987
2020-08-05: 964.3975707144803
2020-08-06: 596.8281091696656
2020-08-07: 877.3426082874505
2020-08-08: 931.5257412865399
2020-08-09: 906.0465933864889
2020-08-10: 1023.4201911643877
2020-08-11: 1012.6634567161329
2020-08-12: 917.0348935670293
2020-08-13: 1034.9541749749183
2020-08-14: 1082.881008147001
2020-08-15: 1107.5094025033893
2020-08-16: 1167.6

2020-08-01: 86.56843495590346
2020-08-02: 110.42804782146678
2020-08-03: 98.01058271719789
2020-08-04: 101.54880665611338
2020-08-05: 105.95280406681528
2020-08-06: 113.35263307855348
2020-08-07: 158.05224323895197
2020-08-08: 177.49737379457397
2020-08-09: 180.56743755645653
2020-08-10: 188.3460326577213
2020-08-11: 197.89421567709252
2020-08-12: 214.89760694537352
2020-08-13: 244.04483159015422
2020-08-14: 262.2630685117373
2020-08-15: 272.92511860932126
2020-08-16: 284.9356103001114
2020-08-17: 299.2348967046573
2020-08-18: 324.6612511144665
2020-08-19: 349.7141104076634
2020-08-20: 369.0647945112083
2020-08-21: 384.90322389925353
2020-08-22: 401.3197925698957
2020-08-23: 430.4133167233808
2020-08-24: 457.0742069538958
2020-08-25: 482.2577647031077
2020-08-26: 504.6074843047414
2020-08-27: 516.7290176455616
2020-08-28: 537.6520346876541
2020-08-29: 564.9017673818685
2020-08-30: 591.8646356370637
2020-08-31: 617.8276985796165

Predicting for Zimbabwe__nan
2020-08-01: 91.1317371375112

In [23]:
# Check the predictions
preds_df.head()

,CountryName,RegionName,Date,PredictedDailyNewCases
213,Aruba,NaN,2020-08-01,58.834154
214,Aruba,NaN,2020-08-02,71.325152
215,Aruba,NaN,2020-08-03,78.992903
216,Aruba,NaN,2020-08-04,90.241488
217,Aruba,NaN,2020-08-05,87.814744


# Validation
This is how the predictor is going to be called during the competition.  
!!! PLEASE DO NOT CHANGE THE API !!!

In [24]:
!python predict.py -s 2020-08-01 -e 2020-08-04 -ip data/2020-09-30_historical_ip_new.csv -o predictions/2020-08-01_2020-08-04.csv

Generating predictions from 2020-08-01 to 2020-08-04...
Saved predictions to predictions/2020-08-01_2020-08-04.csv
Done!


In [25]:
!head predictions/2020-08-01_2020-08-04.csv

'head' 不是内部或外部命令，也不是可运行的程序
或批处理文件。


# Test cases
We can generate a prediction file. Let's validate a few cases...

In [26]:
import os
from covid_xprize.validation.predictor_validation import validate_submission

def validate(start_date, end_date, ip_file, output_file):
    # First, delete any potential old file
    try:
        os.remove(output_file)
    except OSError:
        pass
    
    # Then generate the prediction, calling the official API
    !python predict.py -s {start_date} -e {end_date} -ip {ip_file} -o {output_file}
    
    # And validate it
    errors = validate_submission(start_date, end_date, ip_file, output_file)
    if errors:
        for error in errors:
            print(error)
    else:
        print("All good!")

ModuleNotFoundError: No module named 'covid_xprize'

## 4 days, no gap
- All countries and regions
- Official number of cases is known up to start_date
- Intervention Plans are the official ones

In [ ]:
validate(start_date="2020-08-01",
         end_date="2020-08-04",
         ip_file="../../../validation/data/2020-09-30_historical_ip.csv",
         output_file="predictions/val_4_days.csv")

## 1 month in the future
- 2 countries only
- there's a gap between date of last known number of cases and start_date
- For future dates, Intervention Plans contains scenarios for which predictions are requested to answer the question: what will happen if we apply these plans?

In [ ]:
%%time
validate(start_date="2021-01-01",
         end_date="2021-01-31",
         ip_file="../../../validation/data/future_ip.csv",
         output_file="predictions/val_1_month_future.csv")

## 180 days, from a future date, all countries and regions
- Prediction start date is 1 week from now. (i.e. assuming submission date is 1 week from now)  
- Prediction end date is 6 months after start date.  
- Prediction is requested for all available countries and regions.  
- Intervention plan scenario: freeze last known intervention plans for each country and region.  

As the number of cases is not known yet between today and start date, but the model relies on them, the model has to predict them in order to use them.  
This test is the most demanding test. It should take less than 1 hour to generate the prediction file.

### Generate the scenario

In [ ]:
from datetime import datetime, timedelta

start_date = datetime.now() + timedelta(days=7)
start_date_str = start_date.strftime('%Y-%m-%d')
end_date = start_date + timedelta(days=180)
end_date_str = end_date.strftime('%Y-%m-%d')
print(f"Start date: {start_date_str}")
print(f"End date: {end_date_str}")

In [ ]:
from covid_xprize.validation.scenario_generator import get_raw_data, generate_scenario, NPI_COLUMNS
DATA_FILE = 'data/OxCGRT_latest.csv'
latest_df = get_raw_data(DATA_FILE, latest=True)
scenario_df = generate_scenario(start_date_str, end_date_str, latest_df, countries=None, scenario="Freeze")
scenario_file = "predictions/180_days_future_scenario.csv"
scenario_df.to_csv(scenario_file, index=False)
print(f"Saved scenario to {scenario_file}")

### Check it

In [ ]:
%%time
validate(start_date=start_date_str,
         end_date=end_date_str,
         ip_file=scenario_file,
         output_file="predictions/val_6_month_future.csv")